# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 3
iteration = "-5"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best_test


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2500
loss: 0.16363611817359924
---------------
Batch 2/2500
loss: 0.17939193546772003
---------------
Batch 3/2500
loss: 0.26429903507232666
---------------
Batch 4/2500
loss: 0.16247627139091492
---------------
Batch 5/2500
loss: 0.1818840056657791
---------------
Batch 6/2500
loss: 0.14817309379577637
---------------
Batch 7/2500
loss: 0.1404498666524887
---------------
Batch 8/2500
loss: 0.1573510766029358
---------------
Batch 9/2500
loss: 0.19669590890407562
---------------
Batch 10/2500
loss: 0.4864850640296936
---------------
Batch 11/2500
loss: 0.23708412051200867
---------------
Batch 12/2500
loss: 0.23930826783180237
---------------
Batch 13/2500
loss: 0.161459282040596
---------------
Batch 14/2500
loss: 0.18015310168266296
---------------
Batch 15/2500
loss: 0.21849174797534943
---------------
Batch 16/2500
loss: 0.2683388292789459
---------------
Batch 17/2500
loss: 0.2126757800579071
---------------
Batch 18/2500
loss: 0.1896633803844452
---------------


---------------
Batch 148/2500
loss: 0.13125938177108765
---------------
Batch 149/2500
loss: 0.2251894325017929
---------------
Batch 150/2500
loss: 0.16018365323543549
---------------
Batch 151/2500
loss: 0.10383768379688263
---------------
Batch 152/2500
loss: 0.13960832357406616
---------------
Batch 153/2500
loss: 0.1448858380317688
---------------
Batch 154/2500
loss: 0.1700265258550644
---------------
Batch 155/2500
loss: 0.2020505666732788
---------------
Batch 156/2500
loss: 0.1186935156583786
---------------
Batch 157/2500
loss: 0.2511751651763916
---------------
Batch 158/2500
loss: 0.10099013894796371
---------------
Batch 159/2500
loss: 0.2011394202709198
---------------
Batch 160/2500
loss: 0.1492098569869995
---------------
Batch 161/2500
loss: 0.18335112929344177
---------------
Batch 162/2500
loss: 0.10975529998540878
---------------
Batch 163/2500
loss: 0.1317211091518402
---------------
Batch 164/2500
loss: 0.14449141919612885
---------------
Batch 165/2500
loss: 0.1

loss: 0.20913127064704895
---------------
Batch 293/2500
loss: 0.13407766819000244
---------------
Batch 294/2500
loss: 0.18420915305614471
---------------
Batch 295/2500
loss: 0.19509391486644745
---------------
Batch 296/2500
loss: 0.11067195236682892
---------------
Batch 297/2500
loss: 0.219570592045784
---------------
Batch 298/2500
loss: 0.12986353039741516
---------------
Batch 299/2500
loss: 0.16374316811561584
---------------
Batch 300/2500
loss: 0.1414053738117218
---------------
Batch 301/2500
loss: 0.133401021361351
---------------
Batch 302/2500
loss: 0.10549154877662659
---------------
Batch 303/2500
loss: 0.16841073334217072
---------------
Batch 304/2500
loss: 0.1493716686964035
---------------
Batch 305/2500
loss: 0.12388058006763458
---------------
Batch 306/2500
loss: 0.14246049523353577
---------------
Batch 307/2500
loss: 0.23231768608093262
---------------
Batch 308/2500
loss: 0.13638964295387268
---------------
Batch 309/2500
loss: 0.22998236119747162
-----------

Batch 437/2500
loss: 0.15594099462032318
---------------
Batch 438/2500
loss: 0.1651083379983902
---------------
Batch 439/2500
loss: 0.17652547359466553
---------------
Batch 440/2500
loss: 0.16629314422607422
---------------
Batch 441/2500
loss: 0.22694523632526398
---------------
Batch 442/2500
loss: 0.3093695044517517
---------------
Batch 443/2500
loss: 0.2384095937013626
---------------
Batch 444/2500
loss: 0.18562175333499908
---------------
Batch 445/2500
loss: 0.14384616911411285
---------------
Batch 446/2500
loss: 0.4367799460887909
---------------
Batch 447/2500
loss: 0.16672413051128387
---------------
Batch 448/2500
loss: 0.19132891297340393
---------------
Batch 449/2500
loss: 0.25699466466903687
---------------
Batch 450/2500
loss: 0.17015999555587769
---------------
Batch 451/2500
loss: 0.1423850655555725
---------------
Batch 452/2500
loss: 0.15949484705924988
---------------
Batch 453/2500
loss: 0.4798395335674286
---------------
Batch 454/2500
loss: 0.16257387399673

---------------
Batch 582/2500
loss: 0.33752188086509705
---------------
Batch 583/2500
loss: 0.20697127282619476
---------------
Batch 584/2500
loss: 0.2189982533454895
---------------
Batch 585/2500
loss: 0.26712772250175476
---------------
Batch 586/2500
loss: 0.1122245118021965
---------------
Batch 587/2500
loss: 0.1880800724029541
---------------
Batch 588/2500
loss: 0.15239955484867096
---------------
Batch 589/2500
loss: 0.22680971026420593
---------------
Batch 590/2500
loss: 0.189854234457016
---------------
Batch 591/2500
loss: 0.20798054337501526
---------------
Batch 592/2500
loss: 0.1878887414932251
---------------
Batch 593/2500
loss: 0.18593242764472961
---------------
Batch 594/2500
loss: 0.17251966893672943
---------------
Batch 595/2500
loss: 0.2965870797634125
---------------
Batch 596/2500
loss: 0.14387594163417816
---------------
Batch 597/2500
loss: 0.250409334897995
---------------
Batch 598/2500
loss: 0.12842532992362976
---------------
Batch 599/2500
loss: 0.1

...

---------------
Batch 1822/2500
loss: 0.0699605643749237
---------------
Batch 1823/2500
loss: 0.07811333984136581
---------------
Batch 1824/2500
loss: 0.08514605462551117
---------------
Batch 1825/2500
loss: 0.060012780129909515
---------------
Batch 1826/2500
loss: 0.05822537839412689
---------------
Batch 1827/2500
loss: 0.031416017562150955
---------------
Batch 1828/2500
loss: 0.06452783942222595
---------------
Batch 1829/2500
loss: 0.1025819480419159
---------------
Batch 1830/2500
loss: 0.03613245114684105
---------------
Batch 1831/2500
loss: 0.1039959266781807
---------------
Batch 1832/2500
loss: 0.060650624334812164
---------------
Batch 1833/2500
loss: 0.0660989060997963
---------------
Batch 1834/2500
loss: 0.08128587901592255
---------------
Batch 1835/2500
loss: 0.1390984058380127
---------------
Batch 1836/2500
loss: 0.04988670349121094
---------------
Batch 1837/2500
loss: 0.05144946649670601
---------------
Batch 1838/2500
loss: 0.08040015399456024
---------------


---------------
Batch 1964/2500
loss: 0.23128820955753326
---------------
Batch 1965/2500
loss: 0.28759756684303284
---------------
Batch 1966/2500
loss: 0.1680573970079422
---------------
Batch 1967/2500
loss: 0.17270760238170624
---------------
Batch 1968/2500
loss: 0.270510196685791
---------------
Batch 1969/2500
loss: 0.21031206846237183
---------------
Batch 1970/2500
loss: 0.22857901453971863
---------------
Batch 1971/2500
loss: 0.17297714948654175
---------------
Batch 1972/2500
loss: 0.22464992105960846
---------------
Batch 1973/2500
loss: 0.240974560379982
---------------
Batch 1974/2500
loss: 0.21094666421413422
---------------
Batch 1975/2500
loss: 0.3384339511394501
---------------
Batch 1976/2500
loss: 0.2831456661224365
---------------
Batch 1977/2500
loss: 0.14145204424858093
---------------
Batch 1978/2500
loss: 0.15635201334953308
---------------
Batch 1979/2500
loss: 0.17152956128120422
---------------
Batch 1980/2500
loss: 0.1858994960784912
---------------
Batch 

Batch 2106/2500
loss: 0.11350539326667786
---------------
Batch 2107/2500
loss: 0.1031181737780571
---------------
Batch 2108/2500
loss: 0.08710018545389175
---------------
Batch 2109/2500
loss: 0.08283069729804993
---------------
Batch 2110/2500
loss: 0.12850093841552734
---------------
Batch 2111/2500
loss: 0.1463213711977005
---------------
Batch 2112/2500
loss: 0.11971456557512283
---------------
Batch 2113/2500
loss: 0.1046397015452385
---------------
Batch 2114/2500
loss: 0.08751903474330902
---------------
Batch 2115/2500
loss: 0.07011833041906357
---------------
Batch 2116/2500
loss: 0.11894220858812332
---------------
Batch 2117/2500
loss: 0.12010573595762253
---------------
Batch 2118/2500
loss: 0.10210894793272018
---------------
Batch 2119/2500
loss: 0.08580397814512253
---------------
Batch 2120/2500
loss: 0.10231151431798935
---------------
Batch 2121/2500
loss: 0.10572431236505508
---------------
Batch 2122/2500
loss: 0.12359043955802917
---------------
Batch 2123/2500
l

Batch 2248/2500
loss: 0.15634237229824066
---------------
Batch 2249/2500
loss: 0.13542024791240692
---------------
Batch 2250/2500
loss: 0.1196785643696785
---------------
Batch 2251/2500
loss: 0.09327976405620575
---------------
Batch 2252/2500
loss: 0.1766306459903717
---------------
Batch 2253/2500
loss: 0.09452439099550247
---------------
Batch 2254/2500
loss: 0.12714162468910217
---------------
Batch 2255/2500
loss: 0.106283038854599
---------------
Batch 2256/2500
loss: 0.10407526046037674
---------------
Batch 2257/2500
loss: 0.14293836057186127
---------------
Batch 2258/2500
loss: 0.11161473393440247
---------------
Batch 2259/2500
loss: 0.10486610978841782
---------------
Batch 2260/2500
loss: 0.11244262754917145
---------------
Batch 2261/2500
loss: 0.16519856452941895
---------------
Batch 2262/2500
loss: 0.10479097813367844
---------------
Batch 2263/2500
loss: 0.09861542284488678
---------------
Batch 2264/2500
loss: 0.0808144211769104
---------------
Batch 2265/2500
los

---------------
Batch 2391/2500
loss: 0.1903754323720932
---------------
Batch 2392/2500
loss: 0.10376246273517609
---------------
Batch 2393/2500
loss: 0.10628656297922134
---------------
Batch 2394/2500
loss: 0.11885770410299301
---------------
Batch 2395/2500
loss: 0.09995069354772568
---------------
Batch 2396/2500
loss: 0.2902693748474121
---------------
Batch 2397/2500
loss: 0.13964910805225372
---------------
Batch 2398/2500
loss: 0.1568775773048401
---------------
Batch 2399/2500
loss: 0.18292030692100525
---------------
Batch 2400/2500
loss: 0.1319042295217514
---------------
Batch 2401/2500
loss: 0.13068275153636932
---------------
Batch 2402/2500
loss: 0.1460881233215332
---------------
Batch 2403/2500
loss: 0.14925634860992432
---------------
Batch 2404/2500
loss: 0.11594855040311813
---------------
Batch 2405/2500
loss: 0.13046477735042572
---------------
Batch 2406/2500
loss: 0.14273333549499512
---------------
Batch 2407/2500
loss: 0.09994985163211823
---------------
Bat

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [2]:
import torch
import torch.nn as nn

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(